In [2]:
from IPython.core.display import display     # importing packages
from matplotlib import pyplot as plt
from numpy import linspace
from pandas import DataFrame
import pyomo.environ as pe
import pyomo.opt as po

m = pe.ConcreteModel()

## COST VARIABLES

C_fuel = 5e-6
C_HP = 3e-6
C_LP = 1.8e-6
C_CW = 7e-7

## CONTINUOUS VARIABLES
# in kw
m.Q_fuel = pe.Var(domain=pe.NonNegativeReals) # fuel (input)
m.Q_HP = pe.Var(domain=pe.NonNegativeReals)   # high-pressure steam (input)
m.Q_LP = pe.Var(domain=pe.NonNegativeReals)   # low-pressure steam (input)
m.Q_CW = pe.Var(domain=pe.NonNegativeReals)   # cooling water (input)

m.R1 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 1
m.R2 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 2
m.R3 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 3
m.R4 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 4
m.R5 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 5
m.R6 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 6
m.R7 = pe.Var(domain=pe.NonNegativeReals)   # residual from interval 7

## CONSTRAINTS
def h_balance_1(m):
    return m.R1 - m.Q_fuel == -1500
m.h_balance_1 = pe.Constraint(rule = h_balance_1)

def h_balance_2(m):
    return m.R2 - m.R1 == 900
m.h_balance_2 = pe.Constraint(rule = h_balance_2)

def h_balance_3(m):
    return m.R3 - m.R2 == 500 + m.Q_HP
m.h_balance_3 = pe.Constraint(rule = h_balance_3)

def h_balance_4(m):
    return m.R4 - m.R3 == 2000
m.h_balance_4 = pe.Constraint(rule = h_balance_4)

def h_balance_5(m):
    return m.R5 - m.R4 == -4800 + m.Q_LP
m.h_balance_5 = pe.Constraint(rule = h_balance_5)

def h_balance_6(m):
    return m.R6 - m.R5 == -700
m.h_balance_6 = pe.Constraint(rule = h_balance_6)

def h_balance_7(m):
    return m.R7 - m.R6 == 960
m.h_balance_7 = pe.Constraint(rule = h_balance_7)

def h_balance_8(m):
    return m.Q_CW - m.R7 == 2040
m.h_balance_8 = pe.Constraint(rule = h_balance_8)

def HP_upper(m):
    return m.Q_HP <= 1000
m.HP_upper = pe.Constraint(rule = HP_upper)

def LP_upper(m):
    return m.Q_LP <= 500
m.LP_upper = pe.Constraint(rule = LP_upper)

## OBJECTIVE FUNCTION
def objective_rule_cost(m):
    SYSTEM_COST = (C_fuel*m.Q_fuel + C_HP*m.Q_HP + C_LP*m.Q_LP + C_CW*m.Q_CW) * 3600 * 8000
    return SYSTEM_COST
m.objective = pe.Objective(rule=objective_rule_cost, sense=pe.minimize)

solver = po.SolverFactory('glpk')
solution = solver.solve(m)

m.objective()







475200.00000000006

In [8]:
print(' --- MINIMIZED OBJECTIVE --- ')
print('TOTAL SYSTEM COST:', round(m.objective(), 2), 'USD \n')
print(' --- HEATING AND COOLING MIX --- ')
print('Q_FUEL:', (m.Q_fuel.value), 'kw')
print('Q_HP:', m.Q_HP.value, 'kw')
print('Q_LP:', m.Q_LP.value, 'kw')
print('Q_CW:', m.Q_CW.value, 'kw' '\n')
print(' --- INTERVAL VALUES --- ')
print('R1:', m.R1.value, 'kw')
print('R2:', m.R2.value, 'kw')
print('R3:', m.R3.value, 'kw')
print('R4:', m.R4.value, 'kw')
print('R5:', m.R5.value, 'kw')
print('R6:', m.R6.value, 'kw')
print('R7:', m.R7.value, 'kw')





 --- MINIMIZED OBJECTIVE --- 
TOTAL SYSTEM COST: 475200.0 USD 

 --- HEATING AND COOLING MIX --- 
Q_FUEL: 2100.0 kw
Q_HP: 1000.0 kw
Q_LP: 500.0 kw
Q_CW: 3000.0 kw

 --- INTERVAL VALUES --- 
R1: 600.0 kw
R2: 1500.0 kw
R3: 3000.0 kw
R4: 5000.0 kw
R5: 700.0 kw
R6: 0.0 kw
R7: 960.0 kw


In [3]:
m.display()

Model unknown

  Variables:
    Q_fuel : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2100.0 :  None : False : False : NonNegativeReals
    Q_HP : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1000.0 :  None : False : False : NonNegativeReals
    Q_LP : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 500.0 :  None : False : False : NonNegativeReals
    Q_CW : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 3000.0 :  None : False : False : NonNegativeReals
    R1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 600.0 :  None : False : False : NonNegativeReals
    R2 : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1500.0 :  None : False : False :